In [2]:
#SCP DATA import from db3 file created w/ ROS2 bag
import os
import pandas as pd
import sqlite3

def load_data(path):
    con = sqlite3.connect(path)
    return pd.read_sql_query("SELECT * from messages", con)

rosbagname = "my_test_bag"

db_loc = os.path.join(os.path.expanduser('~'), rosbagname,rosbagname+"_0.db3")

df = load_data(db_loc)

#Regex method to convert all the byte strings to data
df["decode"] = df["data"].str.decode("utf-8")
#regex string to pick all the numbers after "Sensor: " creating a new dataframe
df2 = df["decode"].str.extract("Temperature:\s([\-0-9\.]+).*TDS:\s([\-0-9\.]+).*Voltage:\s([\-0-9\.]+).*Acidity:\s([\-0-9\.]+)",expand = True)
#rename the columns
df2.columns = ["temp","tds","turb","ph"]
#convert to floats
df2 = df2[df2.columns].apply(pd.to_numeric)
#add the timestamp list
df2["timestamp"] = df["timestamp"]



In [4]:
os.environ["GDAL_DATA"] = '/home/pop/.conda/envs/interpol/lib/python3.8/site-packages/rasterio/gdal_data'

In [3]:
import numpy as np
import geopandas as gpd

#Create fake Lat and Long data
df2["lat"] = (np.random.standard_normal(len(df2))/1000)+52
df2["lon"] = (np.random.standard_normal(len(df2))/1000)+5

#create some fake test data as well
df2["test"] = np.random.rand(len(df2))

#create gdf to convert WGS84 to UTM
gdf = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.lon, df2.lat), crs = "EPSG:4326")


In [20]:
#The simplest interpolation function, returns the data and the linspaces

def interkrige(lon,lat,var,res):
    from pykrige.ok import OrdinaryKriging
    import numpy as np

    #GOtta do smth about these lines to create a bit larger of a grid
    x = np.linspace(start = min(lon),stop = max(lon), num = res,retstep = True)
    y = np.linspace(start = min(lat), stop = max(lat), num = res,retstep = True)
    
    gridx = np.linspace(start = min(lon)-x[1], stop = max(lon)+x[1],num = res)
    gridy = np.linspace(start = min(lat)-y[1], stop = max(lat)+y[1],num = res)

    z,SS = OrdinaryKriging(x = lon,
                           y = lat, 
                           z = var, 
                           variogram_model = "linear",
                           coordinates_type = "geographic").execute("grid",xpoints = gridx, ypoints = gridy)
    return (z,SS,gridx,gridy)


In [21]:
#run it
tds_z, tds_SS,x,y = interkrige(df2["lon"], df2["lat"], df2["test"], res=100)

In [ ]:
## Transform it to rasterdata! via rasterio
import rasterio

#projstrings

rdnew = "+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel +towgs84=565.417,50.3319,465.552,-0.398957,0.343988,-1.8774,4.0725 +units=m +no_defs ",

wgs84 = "+proj=longlat +datum=WGS84 +no_defs"

transform = rasterio.transform.guard_transform(rasterio.transform.from_bounds(min(x), min(y), max(x), max(y), 100,100))

new_dataset = rasterio.open(
    "./data/test.tif",
    "w",
    driver = "GTiff",
    height = 100,
    width = 100,
    count = 1,
    dtype =  tds_z.dtype,
    crs= wgs84,
    transform = transform
)

new_dataset.write(tds_z,1)
new_dataset.close()

In [40]:
import folium
import numpy as np
import rasterio 
import geopandas
import matplotlib


r =  rasterio.open("./data/test.tif")
data = r.read(1)
bounds = r.bounds


#Reshape data to form min = 0 and max =1
image = (data - np.min(data))/np.ptp(data)


#set up base map
my_coords = (np.mean(gdf.lat),np.mean(gdf.lon))
m = folium.Map(location = my_coords, zoom_start =17)

#create points from the GPS input data
latitudes = gdf.lat
longitudes = gdf.lon

for lat, lng in zip(latitudes, longitudes):
    folium.Marker(
      location = [lat, lng], 
      icon = folium.Icon(color='red', icon='info-sign')
     ).add_to(m) 

#show the interpolated inputs
folium.raster_layers.ImageOverlay(
    image=image,
    bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
    opacity=1,
    origin='lower',
    colormap = matplotlib.pyplot.cm.Greens
).add_to(m)

m.save(os.path.join('html_folium', 'test.html'))